In [82]:
import pandas as pd
from sqlalchemy import create_engine

In [83]:
crime_file = "../ETLResources/crime.csv"
offense_codes_file = "../ETLResources/offense_codes.csv"

In [84]:
crime_df = pd.read_csv(crime_file)
crime_df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [81]:
offense_codes_df = pd.read_csv(offense_codes_file)
offense_codes_df.head()

,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_TYPE_NAME,OFFENSE_CATEGORY_ID,OFFENSE_CATEGORY_NAME,IS_CRIME,IS_TRAFFIC
0,2804,1,stolen-property-possession,Possession of stolen property,all-other-crimes,All Other Crimes,1,0
1,2804,2,fraud-possess-financial-device,Possession of a financial device,all-other-crimes,All Other Crimes,1,0
2,2901,0,damaged-prop-bus,Damaged business property,public-disorder,Public Disorder,1,0
3,2902,0,criminal-mischief-private,Criminal mischief to private property,public-disorder,Public Disorder,1,0
4,2903,0,criminal-mischief-public,Criminal mischief to public property,public-disorder,Public Disorder,1,0


In [ ]:
crime_df.drop(columns={'GEO_X', 'GEO_Y', 'GEO_LON', 'GEO_LAT', 'OFFENSE_TYPE_ID'})

In [57]:
offense_codes_df['OFFENSE_CODE_EXTENSION']=offense_codes_df['OFFENSE_CODE']\
.map(str) + offense_codes_df['OFFENSE_CODE_EXTENSION'].map(str)

offense_codes_df.drop(columns={'OFFENSE_CODE','OFFENSE_TYPE_ID', 'OFFENSE_CATEGORY_ID'}, inplace=True)
offense_codes_df.head()


,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_NAME,OFFENSE_CATEGORY_NAME,IS_CRIME,IS_TRAFFIC
0,28041,Possession of stolen property,All Other Crimes,1,0
1,28042,Possession of a financial device,All Other Crimes,1,0
2,29010,Damaged business property,Public Disorder,1,0
3,29020,Criminal mischief to private property,Public Disorder,1,0
4,29030,Criminal mischief to public property,Public Disorder,1,0


In [25]:
crime_time_table = crime_df[['INCIDENT_ID', 'OFFENSE_ID', 'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'REPORTED_DATE']].copy()

In [44]:
crime_time_table.rename(columns={'INCIDENT_ID':'incident_id', 
                                 'OFFENSE_ID':'offense_id',
                                 'FIRST_OCCURRENCE_DATE':'first_occurrence',
                                 'LAST_OCCURRENCE_DATE':'last_occurrence',
                                 'REPORTED_DATE':'reported_date'
                                }, inplace=True)

crime_time_table.head()

,incident_id,offense_id,first_occurrence,last_occurrence,reported_date
0,2016376978,2016376978521300,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM
1,20186000994,20186000994239900,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM
2,20166003953,20166003953230500,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM
3,201872333,201872333239900,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM
4,2017411405,2017411405230300,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM


In [27]:
crime_place_table = crime_df[['INCIDENT_ID', 'OFFENSE_ID', 'INCIDENT_ADDRESS', 'NEIGHBORHOOD_ID']].copy()

In [43]:
crime_place_table['DISTRICT_PRECINCT_ID'] = crime_df['DISTRICT_ID'].map(str) + crime_df['PRECINCT_ID'].map(str)
crime_place_table.head()

,INCIDENT_ID,OFFENSE_ID,INCIDENT_ADDRESS,NEIGHBORHOOD_ID,DISTRICT_PRECINCT_ID
0,2016376978,2016376978521300,NaN,montbello,5521
1,20186000994,20186000994239900,NaN,gateway-green-valley-ranch,5522
2,20166003953,20166003953230500,2932 S JOSEPHINE ST,wellshire,3314
3,201872333,201872333239900,705 S COLORADO BLVD,belcaro,3312
4,2017411405,2017411405230300,2810 E 1ST AVE,cherry-creek,3311


In [45]:
rds_connection_string = "postgres:postgres@localhost:5432/denver_crime_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [46]:
engine.table_names()

['crime_time_table', 'crime_place_table', 'offense_codes', 'crime_table']

In [ ]:
crime_time_table.to_sql(name='crime_time_table', con=engine, if_exists='append', index=False)

In [49]:
crime_place_table.rename(columns={'INCIDENT_ID':'incident_id',
                                  'OFFENSE_ID':'offense_id',
                                  'INCIDENT_ADDRESS':'address',
                                  'NEIGHBORHOOD_ID':'neighborhood',
                                  'DISTRICT_PRECINCT_ID':'district_precinct_id'
    
}, inplace=True)
crime_place_table.head()

,incident_id,offense_id,address,neighborhood,district_precinct_id
0,2016376978,2016376978521300,NaN,montbello,5521
1,20186000994,20186000994239900,NaN,gateway-green-valley-ranch,5522
2,20166003953,20166003953230500,2932 S JOSEPHINE ST,wellshire,3314
3,201872333,201872333239900,705 S COLORADO BLVD,belcaro,3312
4,2017411405,2017411405230300,2810 E 1ST AVE,cherry-creek,3311


In [ ]:
crime_place_table.to_sql(name='crime_place_table', con=engine, if_exists='append', index=False)

In [59]:
offense_codes_df.rename(columns={'OFFENSE_CODE_EXTENSION':'offense_code_ext',
                                 'OFFENSE_TYPE_NAME':'offense_name',
                                 'OFFENSE_CATEGORY_NAME':'offense_category',
                                 'IS_CRIME':'is_crime',
                                 'IS_TRAFFIC':'is_traffic'
    
}, inplace=True)
offense_codes_df.head()

,offense_code_ext,offense_name,offense_category,is_crime,is_traffic
0,28041,Possession of stolen property,All Other Crimes,1,0
1,28042,Possession of a financial device,All Other Crimes,1,0
2,29010,Damaged business property,Public Disorder,1,0
3,29020,Criminal mischief to private property,Public Disorder,1,0
4,29030,Criminal mischief to public property,Public Disorder,1,0


In [60]:
offense_codes_df.to_sql(name='offense_codes', con=engine, if_exists='append', index=False)

In [70]:
crime_table = crime_df[['INCIDENT_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION', 'IS_CRIME', 'IS_TRAFFIC']].copy()

In [71]:
crime_table['OFFENSE_CODE_EXTENSION'] = crime_table['OFFENSE_CODE'].map(str) + crime_table['OFFENSE_CODE_EXTENSION'].map(str)

In [72]:
crime_table.drop(columns={'OFFENSE_CODE'}, inplace=True)
crime_table.rename(columns={'INCIDENT_ID':'incident_id',
                            'OFFENSE_CODE_EXTENSION':'offense_code_ext',
                            'IS_CRIME':'is_crime',
                            'IS_TRAFFIC':'is_traffic'
    
}, inplace=True)
crime_table.head()

,incident_id,offense_code_ext,is_crime,is_traffic
0,2016376978,52130,1,0
1,20186000994,23990,1,0
2,20166003953,23050,1,0
3,201872333,23990,1,0
4,2017411405,23030,1,0


In [74]:
crime_table.to_sql(name='crime_table', con=engine, if_exists='append', index=False)

In [76]:
pd.read_sql_query('select * from crime_table', con=engine).head()

,incident_id,offense_code_ext,is_crime,is_traffic
0,2016376978,52130,1,0
1,20186000994,23990,1,0
2,20166003953,23050,1,0
3,201872333,23990,1,0
4,2017411405,23030,1,0


In [77]:
pd.read_sql_query('select * from crime_time_table', con=engine).head()

,incident_id,offense_id,first_occurrence,last_occurrence,reported_date
0,2016376978,2016376978521300,2016-06-15 23:31:00,NaT,2016-06-15 23:31:00
1,20186000994,20186000994239900,2017-10-11 12:30:00,2017-10-11 16:55:00,2018-01-29 17:53:00
2,20166003953,20166003953230500,2016-03-04 20:00:00,2016-04-25 08:00:00,2016-04-26 21:02:00
3,201872333,201872333239900,2018-01-30 19:20:00,NaT,2018-01-30 22:29:00
4,2017411405,2017411405230300,2017-06-22 20:53:00,NaT,2017-06-23 16:09:00


In [78]:
pd.read_sql_query('select * from crime_place_table', con=engine).head()

,incident_id,offense_id,address,neighborhood,district_precinct_id
0,2016376978,2016376978521300,None,montbello,5521
1,20186000994,20186000994239900,None,gateway-green-valley-ranch,5522
2,20166003953,20166003953230500,2932 S JOSEPHINE ST,wellshire,3314
3,201872333,201872333239900,705 S COLORADO BLVD,belcaro,3312
4,2017411405,2017411405230300,2810 E 1ST AVE,cherry-creek,3311


In [79]:
pd.read_sql_query('select * from offense_codes', con=engine).head()

,offense_code_ext,offense_name,offense_category,is_crime,is_traffic
0,28041,Possession of stolen property,All Other Crimes,1,0
1,28042,Possession of a financial device,All Other Crimes,1,0
2,29010,Damaged business property,Public Disorder,1,0
3,29020,Criminal mischief to private property,Public Disorder,1,0
4,29030,Criminal mischief to public property,Public Disorder,1,0
